MTA Data Source:http://web.mta.info/developers/turnstile.html


Data dictionary:    
C/A      = Control Area (a block of turnstiles);
UNIT     = Remote Unit for a station;
SCP      = Subunit Channel Position (designates a single turnstile);
STATION  = Station name;
LINENAME = Concatenation of names of lines going through station (e.g. NQR456, BD,...);
DIVISION = A label with six possible values:
          The first three refer to a time when there were three separate subway systems:  
             IRT = Interborough Rapid Transit;  
             BMT = Brooklyn-Manhattan Transit;  
             IND = Independent Subway;  
          The last three are more recent developments:
             PTH = Port Authority Trans-Hudson (PATH);
             RIT = Roosevelt Island Tram;
             SRT = Staten Island Rapid Transit;
DATE     = Date on which data were collected;
TIME     = Time of collection (every four hours);
DESC     = Characterizes the audit event as either:
          REGULAR (normally every four hours), or
          RECOVR AUD (a missed audit that was recovered);
ENTRIES  = State of entries counter at turnstile at given date and time;
EXITS    = State of exits counter at turnstile at given date and time.

In [181]:
import pandas as pd
from datetime import date, timedelta
from datetime import datetime
from dateutil.parser import parse

baseURL = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_"
start_date = date(2019, 9, 21) # 21st September 2019

############## Helper Functions ##############
def string_of_weeks(start_date, number_of_weeks):
    '''
    Returns a list of strings where each string represents a date in "yymmdd" format
    Starting with the "start_date"
    start_date should be a date object
    '''
    #print(start_date.strftime("%y%m%d")) #for testing
    a_week = timedelta(days=7)
    
    list_of_dates = list([start_date.strftime("%y%m%d")])
    
    #Works but not readable
    #list_of_dates += ([(d-week*a_week).strftime("%y%m%d") for week in range(1,number_of_weeks)])
    
    assert number_of_weeks >= 1
    
    for week in range(1, number_of_weeks):
        temp = start_date - week * a_week # a number * timedelta to walk backwards in time
        list_of_dates.append(temp.strftime("%y%m%d"))
    
    return list_of_dates

#print(string_of_weeks(start_date, 52)) #for testing

def create_MTA_dataframe(start_date, number_of_weeks):
    '''
    Returns a panda DataFrame object of MTA turnstile data starting from
    start_date and going back to number_of_weeks
    '''    
    dates_for_downloads = string_of_weeks(start_date, number_of_weeks)
    print(dates_for_downloads)
    list_of_sizes = []
    
    # --- Read the csv file for the first week & strip any whitespaces in the column names
    mta_df = pd.read_csv(baseURL+dates_for_downloads[0]+".txt")
    mta_df.rename(columns = lambda col: col.strip(), inplace=True)
    list_of_sizes.append(mta_df.shape[0])
    #print(type(mta_df)) #for testing
    
    # --- Read the csv file for the rest of the weeks in the dates list
    #    & strip any whitespaces in the column names before appending
    for week in dates_for_downloads[1:]:
        temp = pd.read_csv(baseURL+week+".txt")
        temp.rename(columns = lambda col: col.strip(), inplace=True)
        mta_df = mta_df.append(temp, ignore_index=True)
        #print("Added one dataframen for: " + week+" with rows:"+ str(temp.shape[0]))
        list_of_sizes.append(temp.shape[0])
        
    
    mta_df.info()
    print(list_of_sizes)
    #print(len(list_of_dfs))
    return mta_df

def add_a_key(list_column_names, mta_data):
    """
    Adds a "key" column by concatenating the columns specified in the "list_column_names"
    Example: To specify "C/A" & "STATION" as keys, call this function using ["C/A", "STATION"]
             for "list_column_names"
    
    returns a pandas DataFrame
    """
    #print(list_column_names)  # for testing
    
    # --- Delete the "key" column if exist & create a brand new key column with empty strings
    if "turnstile_key" in list(mta_data.columns):
        mta_data.drop("turnstile_key", axis=1)
    mta_data["turnstile_key"] = ""
    
    # --- If column name correctly specified and it exists, then add its value to the key
    for col in list_column_names:
        try:
            if col in list(mta_data.columns):
                mta_data["turnstile_key"] += mta_data[col]
        except:
            print("Column {} does not exist in the DataFrame".format(col))
            return None
    
    return mta_data

def add_times_columns(mta_data):
    """
    Adds date/time/day of week/AM-PM columns
    Combines the "DATE" and "TIME" fields into a single colummn of "DATETIME" of datetime data type
    
    Retunrs a pandas DataFrame
    """
    
    list_of_cols = ["DATETIME", "DAY_OF_THE_WEEK", "AM/PM"]
    # --- Delete the time columns if they already exist
    for col in list_of_cols:
        if col in list(mta_data.columns):
            mta_data.drop(col, axis=1)    
    
    # --- DATETIME: Uses parser method from dateutil python library
    #     to convert a string into datetime object  
    #
    # Would this work? mta_data["DATETIME"]=dt.to_datetime(mta_data["DATE"]+" "+mta_data["TIME"])
    #
    mta_data["DATETIME"]=[parse(i) for i in (mta_data["DATE"]+" "+mta_data["TIME"])]
    
    # --- DAY_OF_THE_WEEK: Using standard datetime utils from pandas
    #     to get day of the week from a datetime column
    mta_data["DAY_OF_THE_WEEK"]=mta_data["DATETIME"].dt.weekday_name
    
    # --- AM/PM
    #     Gets AM/PM from a datetime field   
    mta_data["AM/PM"]=[datetime.strftime(dtime, "%p") for dtime in mta_data["DATETIME"]]
    
    return mta_data

def add_previous(mta_data):
    """
    Adds the following column data from previous timestamp rows for any turnstile
    "DATETIME" ==> "prev_datetime"
    "TIMESTAMP" ==> "prev_tstamp"
    "ENTRIES" ==> "prev_entries"
    "EXITS" ==> "prev_exits"
    
    'Previous' data is added for multiple columns for data checks and filtering
    """
    #source_prev_cols = ["DATETIME", "tunrstile_key","ENTRIES", "EXITS"]
    
    prev_column_list = ["prev_datetime", "prev_key", 
                        "prev_entries", "prev_exits",
                        "inSeq", "time_step", "DiffEntries"]
    # --- Delete the prev columns if they already exist
    for col in prev_column_list:
        if col in list(mta_data.columns):
            mta_data.drop(col, axis=1)
    '''
    # --- Delete any "prev" columns if exist, ugly version
    if "prev_datetime" in list(mta_data.columns):
        mta_data.drop("prev_datetime")    
    if "prev_key" in list(mta_data.columns):
        mta_data.drop("prev_key")
    if "prev_entries" in list(mta_data.columns):
        mta_data.drop("prev_entries")
    if "prev_exits" in list(mta_data.columns):
        mta_data.drop("prev_exits")
    if "inSeq" in list(mta_data.columns):
        mta_data.drop("inSeq")
    if "time_step" in list(mta_data.columns):
        mta_data.drop("time_step")
    if "DiffEntries" in list(mta_data.columns):
        mta_data.drop("DiffEntries")
    '''
    # --- sort the dataframe first, so you have rows lined up for getting the previous rows
    mta_data.sort_values(by=[ "turnstile_key", "DATETIME"], ascending = [True, True], inplace=True)
    
    mta_data["prev_datetime"] = mta_data["DATETIME"].shift(periods=1)
    mta_data["prev_key"] = mta_data["turnstile_key"].shift(periods=1)
    mta_data["prev_entries"] = mta_data["ENTRIES"].shift(periods=1)
    mta_data["prev_exits"] = mta_data["EXITS"].shift(periods=1)
    mta_data["inSeq"] = (mta_data["prev_datetime"] < mta_data["DATETIME"]) & (mta_data["prev_key"] == mta_data["turnstile_key"])
    mta_data["DiffEntries"]=(mta_data["ENTRIES"]-mta_data["prev_entries"]) * mta_data["inSeq"]
    mta_data["time_step"] = mta_data["DATETIME"] - mta_data["prev_datetime"]
    
    return mta_data
    

In [170]:
# --- Download data, add a key, add time columns 
#%%timeit
#
#
mta_df = create_MTA_dataframe(start_date, 2)
print(list(mta_df.columns))
mta_df_wKey = add_a_key(["C/A", "UNIT", "SCP", "STATION"], mta_df)
print(list(mta_df.columns))
mta_df_wTimestamp = add_times_columns(mta_df_wKey)

['190921', '190914']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410513 entries, 0 to 410512
Data columns (total 11 columns):
C/A         410513 non-null object
UNIT        410513 non-null object
SCP         410513 non-null object
STATION     410513 non-null object
LINENAME    410513 non-null object
DIVISION    410513 non-null object
DATE        410513 non-null object
TIME        410513 non-null object
DESC        410513 non-null object
ENTRIES     410513 non-null int64
EXITS       410513 non-null int64
dtypes: int64(2), object(9)
memory usage: 34.5+ MB
[204928, 205585]
['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']
['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS', 'turnstile_key']


In [171]:
# --- Stats about the data:
#
#
uniq_CAs = mta_df_wTimestamp['C/A'].unique()
uniq_units = mta_df_wTimestamp['UNIT'].unique()
uniq_scps = mta_df_wTimestamp['SCP'].unique()
uniq_stations = mta_df_wTimestamp['STATION'].unique()
uniq_linenames = mta_df_wTimestamp['LINENAME'].unique()
uniq_divis = mta_df_wTimestamp['LINENAME'].unique()
uniq_keys = mta_df_wTimestamp['turnstile_key'].unique()

print("Number of unique C/A: "+str(len(CAs)))
print("Number of unique UNIT: "+str(len(uniq_units)))
print("Number of unique SCP: "+str(len(uniq_scps)))
print("Number of uniaue STATION: "+str(len(uniq_stations)))
print("Number of uniaue LINENAME: "+str(len(uniq_linenames)))
print("Number of uniaue Turnstile key: "+str(len(uniq_keys)))

Number of unique C/A: 746
Number of unique UNIT: 469
Number of unique SCP: 221
Number of uniaue STATION: 378
Number of uniaue LINENAME: 113
Number of uniaue Turnstile key: 4898


In [172]:
mta_df_wTimestamp

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,turnstile_key,DATETIME,DAY_OF_THE_WEEK,AM/PM
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,00:00:00,REGULAR,7198818,2438323,A002R05102-00-0059 ST,2019-09-14 00:00:00,Saturday,AM
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,04:00:00,REGULAR,7198834,2438325,A002R05102-00-0059 ST,2019-09-14 04:00:00,Saturday,AM
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,08:00:00,REGULAR,7198847,2438354,A002R05102-00-0059 ST,2019-09-14 08:00:00,Saturday,AM
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,12:00:00,REGULAR,7198929,2438428,A002R05102-00-0059 ST,2019-09-14 12:00:00,Saturday,PM
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,16:00:00,REGULAR,7199125,2438483,A002R05102-00-0059 ST,2019-09-14 16:00:00,Saturday,PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410508,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/13/2019,05:00:00,REGULAR,5554,420,TRAM2R46900-05-01RIT-ROOSEVELT,2019-09-13 05:00:00,Friday,AM
410509,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/13/2019,09:00:00,REGULAR,5554,420,TRAM2R46900-05-01RIT-ROOSEVELT,2019-09-13 09:00:00,Friday,AM
410510,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/13/2019,13:00:00,REGULAR,5554,420,TRAM2R46900-05-01RIT-ROOSEVELT,2019-09-13 13:00:00,Friday,PM
410511,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/13/2019,17:00:00,REGULAR,5554,420,TRAM2R46900-05-01RIT-ROOSEVELT,2019-09-13 17:00:00,Friday,PM


In [183]:
mta_df_wPrev = add_previous(mta_df_wTimestamp)
mta_df_wPrev

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,...,DATETIME,DAY_OF_THE_WEEK,AM/PM,prev_datetime,prev_key,prev_entries,prev_exits,inSeq,DiffEntries,time_step
204928,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,00:00:00,REGULAR,7190495,...,2019-09-07 00:00:00,Saturday,AM,NaT,NaN,NaN,NaN,False,NaN,NaT
204929,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,04:00:00,REGULAR,7190516,...,2019-09-07 04:00:00,Saturday,AM,2019-09-07 00:00:00,A002R05102-00-0059 ST,7190495.0,2435566.0,True,21.0,04:00:00
204930,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,08:00:00,REGULAR,7190538,...,2019-09-07 08:00:00,Saturday,AM,2019-09-07 04:00:00,A002R05102-00-0059 ST,7190516.0,2435574.0,True,22.0,04:00:00
204931,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,12:00:00,REGULAR,7190676,...,2019-09-07 12:00:00,Saturday,PM,2019-09-07 08:00:00,A002R05102-00-0059 ST,7190538.0,2435603.0,True,138.0,04:00:00
204932,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,16:00:00,REGULAR,7190931,...,2019-09-07 16:00:00,Saturday,PM,2019-09-07 12:00:00,A002R05102-00-0059 ST,7190676.0,2435676.0,True,255.0,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204923,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,05:00:00,REGULAR,5554,...,2019-09-20 05:00:00,Friday,AM,2019-09-20 01:00:00,TRAM2R46900-05-01RIT-ROOSEVELT,5554.0,420.0,True,0.0,04:00:00
204924,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,09:00:00,REGULAR,5554,...,2019-09-20 09:00:00,Friday,AM,2019-09-20 05:00:00,TRAM2R46900-05-01RIT-ROOSEVELT,5554.0,420.0,True,0.0,04:00:00
204925,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,13:00:00,REGULAR,5554,...,2019-09-20 13:00:00,Friday,PM,2019-09-20 09:00:00,TRAM2R46900-05-01RIT-ROOSEVELT,5554.0,420.0,True,0.0,04:00:00
204926,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,17:00:00,REGULAR,5554,...,2019-09-20 17:00:00,Friday,PM,2019-09-20 13:00:00,TRAM2R46900-05-01RIT-ROOSEVELT,5554.0,420.0,True,0.0,04:00:00


In [186]:
Turnstiles = mta_df_wPrev["turnstile_key"].unique()
mta_df_wPrev[mta_df_wPrev["turnstile_key"] == Turnstiles[3231]].loc[:]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,...,DATETIME,DAY_OF_THE_WEEK,AM/PM,prev_datetime,prev_key,prev_entries,prev_exits,inSeq,DiffEntries,time_step
340041,R130,R321,01-00-00,18 ST,1,IRT,09/07/2019,00:00:00,REGULAR,141945,...,2019-09-07 00:00:00,Saturday,AM,2019-09-20 20:00:00,R129R32100-00-0218 ST,15340469.0,4513869.0,False,-0.0,-14 days +04:00:00
340042,R130,R321,01-00-00,18 ST,1,IRT,09/07/2019,04:00:00,REGULAR,141964,...,2019-09-07 04:00:00,Saturday,AM,2019-09-07 00:00:00,R130R32101-00-0018 ST,141945.0,116049.0,True,19.0,0 days 04:00:00
340043,R130,R321,01-00-00,18 ST,1,IRT,09/07/2019,08:00:00,REGULAR,141972,...,2019-09-07 08:00:00,Saturday,AM,2019-09-07 04:00:00,R130R32101-00-0018 ST,141964.0,116060.0,True,8.0,0 days 04:00:00
340044,R130,R321,01-00-00,18 ST,1,IRT,09/07/2019,12:00:00,REGULAR,142043,...,2019-09-07 12:00:00,Saturday,PM,2019-09-07 08:00:00,R130R32101-00-0018 ST,141972.0,116077.0,True,71.0,0 days 04:00:00
340045,R130,R321,01-00-00,18 ST,1,IRT,09/07/2019,16:00:00,REGULAR,142188,...,2019-09-07 16:00:00,Saturday,PM,2019-09-07 12:00:00,R130R32101-00-0018 ST,142043.0,116183.0,True,145.0,0 days 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134740,R130,R321,01-00-00,18 ST,1,IRT,09/20/2019,04:00:00,REGULAR,151487,...,2019-09-20 04:00:00,Friday,AM,2019-09-20 00:00:00,R130R32101-00-0018 ST,151474.0,123686.0,True,13.0,0 days 04:00:00
134741,R130,R321,01-00-00,18 ST,1,IRT,09/20/2019,08:00:00,REGULAR,151526,...,2019-09-20 08:00:00,Friday,AM,2019-09-20 04:00:00,R130R32101-00-0018 ST,151487.0,123693.0,True,39.0,0 days 04:00:00
134742,R130,R321,01-00-00,18 ST,1,IRT,09/20/2019,12:00:00,REGULAR,151776,...,2019-09-20 12:00:00,Friday,PM,2019-09-20 08:00:00,R130R32101-00-0018 ST,151526.0,123789.0,True,250.0,0 days 04:00:00
134743,R130,R321,01-00-00,18 ST,1,IRT,09/20/2019,16:00:00,REGULAR,152032,...,2019-09-20 16:00:00,Friday,PM,2019-09-20 12:00:00,R130R32101-00-0018 ST,151776.0,124040.0,True,256.0,0 days 04:00:00


In [187]:
mta_df_wPrev[mta_df_wPrev["DiffEntries"]<0]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,...,DATETIME,DAY_OF_THE_WEEK,AM/PM,prev_datetime,prev_key,prev_entries,prev_exits,inSeq,DiffEntries,time_step
206525,A011,R080,01-03-00,57 ST-7 AV,NQRW,BMT,09/07/2019,04:00:00,REGULAR,885818868,...,2019-09-07 04:00:00,Saturday,AM,2019-09-07 00:00:00,A011R08001-03-0057 ST-7 AV,8.858189e+08,4.903259e+08,True,-35.0,04:00:00
206526,A011,R080,01-03-00,57 ST-7 AV,NQRW,BMT,09/07/2019,08:00:00,REGULAR,885818856,...,2019-09-07 08:00:00,Saturday,AM,2019-09-07 04:00:00,A011R08001-03-0057 ST-7 AV,8.858189e+08,4.903258e+08,True,-12.0,04:00:00
206527,A011,R080,01-03-00,57 ST-7 AV,NQRW,BMT,09/07/2019,12:00:00,REGULAR,885818776,...,2019-09-07 12:00:00,Saturday,PM,2019-09-07 08:00:00,A011R08001-03-0057 ST-7 AV,8.858189e+08,4.903257e+08,True,-80.0,04:00:00
206528,A011,R080,01-03-00,57 ST-7 AV,NQRW,BMT,09/07/2019,16:00:00,REGULAR,885818662,...,2019-09-07 16:00:00,Saturday,PM,2019-09-07 12:00:00,A011R08001-03-0057 ST-7 AV,8.858188e+08,4.903254e+08,True,-114.0,04:00:00
206529,A011,R080,01-03-00,57 ST-7 AV,NQRW,BMT,09/07/2019,20:00:00,REGULAR,885818530,...,2019-09-07 20:00:00,Saturday,PM,2019-09-07 16:00:00,A011R08001-03-0057 ST-7 AV,8.858187e+08,4.903249e+08,True,-132.0,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203114,R730,R431,00-00-04,EASTCHSTER/DYRE,5,IRT,09/20/2019,09:00:00,REGULAR,1559930418,...,2019-09-20 09:00:00,Friday,AM,2019-09-20 05:00:00,R730R43100-00-04EASTCHSTER/DYRE,1.559931e+09,1.728191e+09,True,-319.0,04:00:00
203115,R730,R431,00-00-04,EASTCHSTER/DYRE,5,IRT,09/20/2019,13:00:00,REGULAR,1559930346,...,2019-09-20 13:00:00,Friday,PM,2019-09-20 09:00:00,R730R43100-00-04EASTCHSTER/DYRE,1.559930e+09,1.728191e+09,True,-72.0,04:00:00
203116,R730,R431,00-00-04,EASTCHSTER/DYRE,5,IRT,09/20/2019,17:00:00,REGULAR,1559930260,...,2019-09-20 17:00:00,Friday,PM,2019-09-20 13:00:00,R730R43100-00-04EASTCHSTER/DYRE,1.559930e+09,1.728191e+09,True,-86.0,04:00:00
203117,R730,R431,00-00-04,EASTCHSTER/DYRE,5,IRT,09/20/2019,21:00:00,REGULAR,1559930207,...,2019-09-20 21:00:00,Friday,PM,2019-09-20 17:00:00,R730R43100-00-04EASTCHSTER/DYRE,1.559930e+09,1.728191e+09,True,-53.0,04:00:00


In [188]:
mta_df_wPrev[mta_df_wPrev["turnstile_key"] == "TRAM1R46800-00-01RIT-MANHATTAN"].to_csv("TramQC.csv")